# Search for GeV illuminated clouds near SNRs

In [1]:
import pyJam.fermi.SNR_MC.coGevOverlap as cogev
import pandas as pd
from pandas.io.json import json_normalize
import os, sys, re
import fnmatch
from glob import glob
import operator
import astropy.io.fits as pyfits
from astropy import units as u
from astropy import constants as c
from astropy.coordinates import SkyCoord
from astropy import wcs
from astropy.table import Table
from astropy.io import ascii
import numpy as np
import yaml
from collections import defaultdict, OrderedDict
import pyJam.utilities.saveResults as sv
from pyJam.fermi.SNR_MC import distNsep as sep
#from skymaps import SkyImage,SkyDir #skymaps only works if I run at SLAC, maybe modify findPeak to use astropy instead
#from uw.like.roi_analysis import ROIAnalysis #pointlike stuff only works at SLAC. Don't need them if I have the yaml?
#from uw.like.roi_state import PointlikeState #pointlike stuff only works at SLAC. Don't need them if I have the yaml?
from lande.utilities.tools import tolist


/Users/elunejava/anaconda/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [10]:
#reload(cogev)
%load_ext autoreload
#%aimport cg
%autoreload 2

In [287]:
#Should I just make an assocGeVCO object and get the files from there to play with
#on my computer
#baseNbyDir = '/Users/jmcohen/Desktop/cur/nbyMaps/P7rep5year3FGL_fixedDiff/3fgl/' #where all the nbyMaps are
#ptlkBaseDir = '/Users/jmcohen/Desktop/cur/P7rep/3degAdd_fixedDiff/' #where all the pointlike results are
baseNbyDir = '/Users/jmcohen/Desktop/cur/nbyMaps/P7rep5year3FGL/3fgl/' #where all the nbyMaps are
ptlkBaseDir = '/Users/jmcohen/Desktop/cur/P7rep/' #where all the pointlike results are #maybe this is the non 3deg fixed version?
FGLCat = '/Users/jmcohen/Dropbox/Catalogs/3FGL/3FGL-v13r3_v6r7p1_3lacv11p9_v3.fits' #the version of 3FGL I used to determine the candiate 3FGL MCs
vel = 5
energyDir = '1gevTo316gev'
#at SLAC
# baseNbyDir = '/afs/slac/g/glast/users/jmcohen/diffusionStudy/nearbySources/P7rep5year3FGL_fixedDiff/3fgl/'
# ptlkBaseDir = '/afs/slac/g/glast/users/jmcohen/P7rep/3degAdd_fixedDiff/'
# FGLCat = '/afs/slac/g/glast/users/jmcohen/st_files/3FGL/3FGL-v13r3_v6r7p1_3lacv11p9_v3.fits'

In [288]:
#have to choose a velocity
#All the 3FGL stuff in coGeV.py is specific to the internal 3FGL cat
#do I need to make changes to it?
cg = cogev.assocGeVCO(baseNbyDir = baseNbyDir,
                      ptlkBaseDir = ptlkBaseDir,
                      FGLCat = FGLCat,
                      vel = 5)

In [289]:
cg.nbyInfo.keys()

['SNR089.0+04.7',
 'SNR127.1+00.5',
 'SNR027.8+00.6',
 'SNR049.2-00.7',
 'SNR006.4-00.1',
 'SNR189.1+03.0',
 'SNR034.7-00.4',
 'SNR132.7+01.3',
 'SNR073.9+00.9',
 'SNR205.5+00.5']

# Get 3FGL
### check if the version I use previously is the same as the current version of the catalog

In [8]:
#Don't use this to get 3FGL cat
#fgl3 = cg.getLAT() #getLAT contains more than just 3FGL, it combines my 
#fgl3OldDF = pd.DataFrame.from_dict(fgl3)
#fgl3OldDF = pd.DataFrame(fgl3) #this does weird things, it only has info for W44 for some reason + the 3FGL info


In [282]:
def getFGL(FGLCat = "/Users/jmcohen/Dropbox/Catalogs/3FGL/gll_psc_v14.fit"):
    """Make pd.DF from 3FGL fits file"""
    fgl3 = pyfits.open(FGLCat)
    fgl3d =  fgl3[1].data
    poplist = []
    fgl3dict = {x:fgl3d.field(x) for x in fgl3d.names}
    for i in fgl3dict:
        fgl3dict[i] = list(fgl3dict.get(i))
    #     if fgl3dict.get(i).ndim > 1:
    #         poplist.append(i)

    # for i in poplist:
    #     fgl3dict.pop(i)
    fgl3DF = pd.DataFrame(fgl3dict)
    #fgl3DF.columns
    if 'gll' in FGLCat:
        fgl3DF.set_index('Source_Name',inplace = True)
    else: #
        fgl3DF.set_index('Source_Name_3FGL',inplace = True)
    maxl = len(fgl3DF)
    #print maxl
    pd.set_option('display.max_columns', maxl)
    pd.set_option('display.max_rows', maxl) #I will transpose the frame later
    return fgl3DF

In [283]:
newFglDF = getFGL()
newFglDF.replace('',np.nan,inplace=True)
oldFglDF = getFGL(FGLCat)
oldFglDF.replace('',np.nan,inplace=True)

In [46]:
#check if 3FGL CLASS are the same in new and old 3FGL fits file
sameClass = newFglDF.CLASS1== oldFglDF.CLASS_3FGL #).all(axis=1).astype(int)
classDF = pd.DataFrame({'new3FGL':newFglDF.CLASS1[~sameClass],'old3FGL':oldFglDF.CLASS_3FGL[~sameClass]})
#classDF = classDF[classDF.new3FGL != 'bcu'] #agu became bcu, remove all
#is new 3FGL bcu the same as old agu? These are the majority, so if these are the same, then I should exlcude them
#HMBX became HMB in new
#newFglDF[newFglDF.CLASS1 == 'agu'] #new 3FGL doesn't have agu
#oldFglDF[oldFglDF.CLASS_3FGL == 'bcu'] #old 3FGL doesn't have bcu
#what I really want are ones where 
##newFGL isuppper and was lower before
##new 3FGL isupper and was blank before

In [ ]:
#don't need this first one
#I only want to retain the case when new3FGL is upper and old3FGL is lowe
#classDF[(classDF.new3FGL != 'bcu') & (classDF.new3FGL !='BCU') & (classDF.new3FGL != 'HMB')]
classDF[(classDF.new3FGL.str.isupper()) & (classDF.old3FGL.str.islower())]

In [60]:
#For all the ones with differnt CLASS, which are ID'd
#only pulsars bumped from assoc to ID
classDF[classDF.new3FGL.str.isupper()]

,new3FGL,old3FGL
Source_Name,,
3FGL J0210.7-5101,BCU,AGU
3FGL J0240.5+6113,HMB,HMXB
3FGL J0522.9-3628,BCU,AGU
3FGL J0532.0-4827,BCU,AGU
3FGL J0621.0+2514,PSR,psr
3FGL J0622.9+3326,BCU,AGU
3FGL J0740.8+6621,PSR,psr
3FGL J0930.9-1904,PSR,psr
3FGL J1018.9-5856,HMB,HMXB


# Get nbyInfo and results from pointlike, put into DF
## cg.ptlkInfo has all the details of the pointlike runs, nbyInfo took some of this but not all
* This is troublesome because it's not trivial to go from nested dict to DF
* Is it easier to make it into a panel?
* Or keep them as dicts and pull them out when I need them?
* make dict and then DF?
* LOOK AT WHAT I DID WITH THAT CONFLUENCE TABLE?

In [18]:
#Not sure this will work ir if I want to use it yet
#https://stackoverflow.com/questions/23019119/converting-multilevel-nested-dictionaries-to-pandas-dataframe
def flatten(d):
    """
    The flatten function creates a new OrderedDict object--the flatten dictionary
    Cycle through each key/value pair
    If a value is not a dictionary (OrderDict included), then copy the key/value pair to the result
    If the value is a dictionary, including OrderedDict, then call flatten recursively and update the result
    I have tested this function with nesting of four levels deep, but cannot guarantee it is bug-free
    Once you tested flatten on one OrderedDict, it is straight-forward to apply it to a list of OrderedDict
"""
    Flatten an OrderedDict object
    '''
    result = OrderedDict()
    for k, v in d.items():
        if isinstance(v, dict):
            result.update(flatten(v))
        else:
            result[k] = v
    return result

In [38]:
# for i in cg.nbyInfo:
#     if isinstance(cg.nbyInfo.get(i),dict):
#         #print i,'FLAT',flatten(cg.nbyInfo.get(i)),'\n'
#         for j in cg.nbyInfo.get(i):
#             if isinstance(cg.nbyInfo.get(i).get(j),dict):
#                 print j, cg.nbyInfo.get(i).get(j)
#             else: print j
#     else:
#         pass
#         #print i,'not flat',cg.nbyInfo(i),'\n'


In [36]:
def dict2Multi(dictionary):
    """try converting the nested dict to a multi-index DF
    https://stackoverflow.com/questions/24988131/nested-dictionary-to-multiindex-dataframe-where-dictionary-keys-are-column-label"""
    
    reform = {(outerKey, innerKey): values for outerKey, innerDict in dictionary.iteritems() for innerKey, values in innerDict.iteritems()}
    return pd.DataFrame(reform)

In [39]:
#this is close but not quite right
#the problem is that some of the values for each SNR have a single value and not a dict
#the problem is that i need it to look like
#SNR
#
nbyDF = dict2Multi(cg.nbyInfo)
#nbyDF


In [40]:
#nbyInfo has all the results from 
#print cg.nbyInfo
#nby = pd.DataFrame.from_dict(cg.nbyInfo)
#json_normalize(pd.DataFrame.from_dict(cg.nbyInfo))
#nby

### I think the 2 table's I want are
* Nby Cand | properties | SNR it's near | For this, I need to rename the PS sources
* Just append the SNR RoI maybe## SNR | SNR properties

In [43]:
# candsDict = dict()
# snrDict = dict()
# for snr,v in cg.nbyInfo.items():
#     for p in v:
#         if p.startswith('PS'):
#             #append SNR to PS for now
#         elif p.startswith('P7R'):
#             #change P7R name to 3FGL name
#         else: #if it's not PS or P7R, then it's a property of the SNR
#             snrDict[] =  
            
# {'SNR': snrList,'50pcAngSep': angPCList,'NickName': nickList,'RadioRadius': radList,'RadioRadiusPC': radPCList,'dist': distList,'ext': extList,'extPC': extList,'snrvel': velList}

In [284]:
snrs = sorted(cg.nbyInfo.keys())
#make SNR DF
angPCList = [round(cg.nbyInfo[i].get('50pcAngSep'),2) for i in iter(sorted(cg.nbyInfo))]
nickList = [cg.nbyInfo[i].get('NickName') for i in iter(sorted(cg.nbyInfo))]
radList = [round(cg.nbyInfo[i].get('RadioRadius'),2) for i in iter(sorted(cg.nbyInfo))]
radPCList = [round(cg.nbyInfo[i].get('RadioRadiusPC'),0) for i in iter(sorted(cg.nbyInfo))]
distList = [round(cg.nbyInfo[i].get('dist'),1) for i in iter(sorted(cg.nbyInfo))]
#extList = [round(cg.nbyInfo[i].get('ext'),2) for i in iter(sorted(cg.nbyInfo))]
extList = [cg.nbyInfo[i].get('ext',np.nan) for i in iter(sorted(cg.nbyInfo))]
#extPCList = [round(cg.nbyInfo[i].get('extPC'),2) for i in iter(sorted(cg.nbyInfo))]
extPCList = [cg.nbyInfo[i].get('extPC',np.NaN) for i in iter(sorted(cg.nbyInfo))]
velList = [round(cg.nbyInfo[i].get('vel'),2)for i in iter(sorted(cg.nbyInfo))]
snrsDF = pd.DataFrame({'50pc Ang Sep': angPCList,'Nick Name': nickList,'Radio  Radius': radList,'Radio Radius PC': radPCList,'Distance': distList,'sigma': extList,'sigma_pc': extPCList,'Velocity': velList},index =  snrs)
snrsDF.index.name = 'SNR'
snrsDF.replace('Point',np.nan,inplace=True)
snrsDF['sigma'] = snrsDF['sigma'].round(2)#.map('{:,.2f}'.format)
snrsDF['sigma_pc'] = snrsDF['sigma_pc'].round(0)#.map('{:,.2f}'.format)
#why are these the only SNRs?
#Is it because pointlike run didn't finish or because nbyMaps?
#probably latter, if so, use pointlike instead and just get this infor from the results of getSNRsample
#Maybe I should just get this
#snrs

In [285]:
#rename columns?
#don't need nickname for paper
#how many signif digits?
#combine ext/extPC?
#footnotes
#a- radioRadius, all from Green's cat
#b- distance ref
#c- vel ref
#I think I should just ref the SNR catalog, I got many of those refs for d and v
snrsDF


,50pc Ang Sep,Distance,Nick Name,Radio Radius,Radio Radius PC,Velocity,sigma,sigma_pc
SNR,,,,,,,,
SNR006.4-00.1,1.51,1.9,W28,0.40,13,10.0,0.38,12
SNR027.8+00.6,0.95,3.0,None,0.32,17,99.5,NaN,NaN
SNR034.7-00.4,0.95,3.0,W44,0.26,14,45.0,0.31,16
SNR049.2-00.7,0.48,6.0,W51c,0.25,26,70.0,0.31,32
SNR073.9+00.9,0.95,3.0,None,0.23,12,4.0,NaN,NaN
SNR089.0+04.7,1.69,1.7,HB21,0.87,26,-10.0,0.99,29
SNR127.1+00.5,2.49,1.1,R5,0.38,8,-14.0,NaN,NaN
SNR132.7+01.3,1.30,2.2,HB3,0.67,26,-43.0,0.82,31
SNR189.1+03.0,1.91,1.5,IC443,0.38,10,-6.0,0.32,8


In [286]:
snrs

['SNR006.4-00.1',
 'SNR027.8+00.6',
 'SNR034.7-00.4',
 'SNR049.2-00.7',
 'SNR073.9+00.9',
 'SNR089.0+04.7',
 'SNR127.1+00.5',
 'SNR132.7+01.3',
 'SNR189.1+03.0',
 'SNR205.5+00.5']

In [141]:
#make Candidate DF
#any that are candidates for >1 SNRs?
#yes! but only the PS 
#I need to 
nonCandPars = ['50pcAngSep','NickName','RadioRadius','RadioRadiusPC','dist','ext','extPC','vel']
#allCands = {j:cg.nbyInfo.get(i).values() for i in snrs for j in cg.nbyInfo.get(i).keys() if not j in nonCandPars}
allCands = {j+'_'+i:{:cg.nbyInfo.get(i).get(j).values()} for i in snrs for j in cg.nbyInfo.get(i).keys() if not j in nonCandPars}
#candsSNR = [c.split('_SNR')[0] for c in allCands]
cands = [c for c in allCands]
#cand_CLASS1 = [allCands.get(p)['CLASS1'] for p in cands]
# cand_TS = [allCands.get(p)['TS'] for p in cands]
# cand_index = [allCands.get(p)['index'] for p in cands]
# cand_indexErr = [allCands.get(p)['indexErr'] for p in cands]
# cand_srcSep = [allCands.get(p)['srcSep'] for p in cands]
# cand_srcSepPC = [allCands.get(p)['srcSepPC'] for p in cands]
#LATInd1 = [i.get('index1',np.nan) if isinstance(i,dict)  else np.nan for i in LATSpecp fofr p in ]

In [241]:
#nbySrcs = []
if 'nbySrcsDF' in locals():
    del nbySrcsDF
nbySrcsDF = pd.DataFrame()
nbySrcsDF = pd.DataFrame(columns=['TS', 'CLASS1', 'srcSepPC', 'srcSep', 'index', 'indexErr'])
TS = []
class1 = []
srcSepPC = []
srcSep = []
index = []
for snr in sorted(snrs): 
    nbySrc = ''
    for src in cg.nbyInfo[snr]:
        if src.startswith('PS') or src.startswith('P7'):
            # class1.append(nbyInfo[snr][src].get('CLASS1'))
            # TS.append(nbyInfo[snr][src].get('TS'))
            # srcSepPC.append(nbyInfo[snr][src].get('srcSepPC.append('))
            # srcSep.append(nbyInfo[snr][src].get('srcSep'))
            TS = int(round(cg.nbyInfo[snr][src].get('TS'),0))
#             if not TS >= 16.:
#                 continue
            class1 = cg.nbyInfo[snr][src].get('CLASS1')
            if class1 == '' or not class1:
                class1 = None
            srcSepPC = round(cg.nbyInfo[snr][src].get('srcSepPC'),2)
            srcSep = round(cg.nbyInfo[snr][src].get('srcSep'),2)
            index = round(cg.nbyInfo[snr][src].get('index'),2)
            indexErr = round(cg.nbyInfo[snr][src].get('indexErr'),2)
            #nbySrc = nbySrc + '%s, TS = %s, srcSepDeg = %s, srcSepPC = %s, Index = %s +/- %s , CLASS1 = %s \\\\ \\\\ ' %(src, TS, srcSep, srcSepPC, index, indexErr,class1)
            nbySrcsDF.loc[src+'_'+snr] = [TS,class1,srcSepPC,srcSep,index,indexErr]
nbySrcsDF.rename(columns={'CLASS1':'ASSOC1'},inplace = True) #CLASS1 should all unID'd


In [301]:
nbySrcsDF

,TS,ASSOC1,srcSepPC,srcSep,index,indexErr
PS_6_SNR034.7-00.4,60,NaN,46.25,0.88,2.42,0.27
P7R42958A_SNR034.7-00.4,120,NaN,49.59,0.95,1.96,0.59
P7R42955_SNR034.7-00.4,320,2FGL J1857.2+0055c,24.88,0.48,1.91,0.11
PS_4_SNR034.7-00.4,74,NaN,27.78,0.53,2.44,0.09
P7R42958_SNR034.7-00.4,211,EGRJ1856+0235 2FGL J1857.6+0211 ...,48.39,0.92,2.46,0.24
PS_2_SNR034.7-00.4,46,NaN,47.56,0.91,2.98,0.26
P7R42955G_SNR034.7-00.4,6,1AGLJ1857+0137 2FGL J1858.5+0129c,24.11,0.46,2.01,0.24
PS_3_SNR049.2-00.7,0,NaN,37.23,0.36,3.71,NaN
P7R43175_SNR073.9+00.9,614,2CG 075+00 2FGL J2018.0+3626,43.59,0.83,2.68,0.17
PS_1_SNR089.0+04.7,34,NaN,36.27,1.22,2.71,0.20


In [257]:
snrTab = Table(np.array(snrsDF))
ascii.write(snrTab, Writer=ascii.AASTex)#,
                 #latexdict = {'units': {'mass': 'kg', 'speed': 'km/h'}})
#
#class is wrong, this is really assoc1 
#This is all determined from an old version of 3FGL, I have a newer version
#replace? the classes should all be unID'd though anyway, so just drop this, or change name to ASSOC1
#add l/b since PS sources are new
#what are the high TS PS sources?
#did I remove things and then readd them?

\begin{deluxetable}{cccccccc}
\tablehead{\colhead{col0} & \colhead{col1} & \colhead{col2} & \colhead{col3} & \colhead{col4} & \colhead{col5} & \colhead{col6} & \colhead{col7}}
\startdata
0.95 & 3.0 & W44 & 0.26 & 14.0 & 45.0 & 0.31 & 16.0 \\
0.48 & 6.0 & W51c & 0.25 & 26.0 & 70.0 & 0.3 & 31.0 \\
0.95 & 3.0 & None & 0.23 & 12.0 & 4.0 & nan & nan \\
1.69 & 1.7 & HB21 & 0.87 & 26.0 & -10.0 & 0.94 & 28.0 \\
2.49 & 1.1 & R5 & 0.38 & 8.0 & -14.0 & nan & nan \\
1.3 & 2.2 & HB3 & 0.67 & 26.0 & -43.0 & 0.81 & 31.0 \\
1.91 & 1.5 & IC443 & 0.38 & 10.0 & -6.0 & 0.32 & 8.0 \\
1.91 & 1.5 & Monoceros & 1.83 & 48.0 & 11.0 & 2.78 & 73.0 \\
\enddata
\end{deluxetable}


## ptlkInfo only has info for single SNR, write something to combine all the yamls
## each yaml has info for every source in the RoI, but I only want the ones that are nby sources
## look at nbyMaps to see what I did there
### in nbyMaps, I actually loaded in the pointlike saved dat and then calculated the distance from there to determine which were nby
### I tried looping over all the ptlk result yamls, but this gives info on all sources in RoI(MC cand and non-cands)
## Why didn't I want to use the results_allNbyInfo.yaml?
### Oh right, because not all of the runs finished for results_allNbyInfo.yaml in each velocity
### I want info on 18 SNRs

In [117]:
#ptlkInfo only has info for single SNR
#write something to combine all the yamls
#analysis for 18 SNRs
os.chdir(ptlkBaseDir)
#ptlkYamlList = [os.path.join(dirpath, f) for dirpath, dirnames, files in os.walk(ptlkBaseDir) if '1gevTo316gev' in dirpath for f in fnmatch.filter(files, 'results_SNR*.yaml' %locals()) if not 'moved' in f]
#here's another way to do this using glb instead of all this walking and fnmatch, much easier I think
ptlkYamlDict = {os.path.split(os.path.split(x)[0])[0]:os.path.join(ptlkBaseDir,x) for x in glob('SNR*/%s/results_SNR*.yaml' %(energyDir)) if not 'moved' in x}


In [119]:
#make a dict with SNR name and contents of each yaml
ptlkAllDict = dict()
for snr in ptlkYamlDict:
    ptlkYamlDict[snr] = 
    

In [271]:
#check if any of the results_allNbyInfo yamls for each vel has all the SNRs
#what other extraneous info is in here? maybe none?
#bdir = baseNbyDir
bdir = "/Users/jmcohen/Desktop/cur/nbyMaps/P7rep5year3FGL/3fgl"
#bdir = "/Users/jmcohen/Desktop/cur/nbyMaps/chkFixedDiff/P7rep5year3FGL_fixedDiff/3fgl/"
vels = [3,5,10,15]
#vels = [3,15]
for v in vels:
    cg = cogev.assocGeVCO(baseNbyDir = bdir,
                      ptlkBaseDir = ptlkBaseDir,
                      FGLCat = FGLCat,
                      vel = v)
    print 'vel', v, len(sorted(cg.nbyInfo.keys())), sorted(cg.nbyInfo.keys())
#Still some missing :(
#which?



vel 3 11 ['SNR006.4-00.1', 'SNR027.8+00.6', 'SNR034.7-00.4', 'SNR049.2-00.7', 'SNR073.9+00.9', 'SNR089.0+04.7', 'SNR127.1+00.5', 'SNR132.7+01.3', 'SNR189.1+03.0', 'SNR205.5+00.5', 'SNR347.3-00.5']
vel 5 10 ['SNR006.4-00.1', 'SNR027.8+00.6', 'SNR034.7-00.4', 'SNR049.2-00.7', 'SNR073.9+00.9', 'SNR089.0+04.7', 'SNR127.1+00.5', 'SNR132.7+01.3', 'SNR189.1+03.0', 'SNR205.5+00.5']
vel 10 10 ['SNR006.4-00.1', 'SNR027.8+00.6', 'SNR034.7-00.4', 'SNR049.2-00.7', 'SNR073.9+00.9', 'SNR089.0+04.7', 'SNR127.1+00.5', 'SNR132.7+01.3', 'SNR189.1+03.0', 'SNR205.5+00.5']
vel 15 11 ['SNR006.4-00.1', 'SNR027.8+00.6', 'SNR034.7-00.4', 'SNR049.2-00.7', 'SNR073.9+00.9', 'SNR089.0+04.7', 'SNR127.1+00.5', 'SNR132.7+01.3', 'SNR189.1+03.0', 'SNR205.5+00.5', 'SNR347.3-00.5']


### how many pointlike runs completed?

In [296]:
#!cd "/Users/jmcohen/Desktop/cur/P7rep"
os.chdir(ptlkBaseDir)
!pwd

/Users/jmcohen/Desktop/cur/P7rep


In [298]:
{os.path.split(os.path.split(x)[0])[0]:os.path.join(ptlkBaseDir,x) for x in glob('SNR*/%s/SNR*.dat' %(energyDir)) if not 'moved' in x}
#glob('SNR*/')#%s/SNR*.dat' %(energyDir))

{'SNR049.2-00.7': '/Users/jmcohen/Desktop/cur/P7rep/SNR049.2-00.7/1gevTo316gev/SNR049.2-00.7.dat',
 'SNR089.0+04.7': '/Users/jmcohen/Desktop/cur/P7rep/SNR089.0+04.7/1gevTo316gev/SNR089.0+04.7.dat',
 'SNR127.1+00.5': '/Users/jmcohen/Desktop/cur/P7rep/SNR127.1+00.5/1gevTo316gev/SNR127.1+00.5_SBPL.dat',
 'SNR132.7+01.3': '/Users/jmcohen/Desktop/cur/P7rep/SNR132.7+01.3/1gevTo316gev/SNR132.7+01.3.dat',
 'SNR189.1+03.0': '/Users/jmcohen/Desktop/cur/P7rep/SNR189.1+03.0/1gevTo316gev/SNR189.1+03.0_SBPL.dat',
 'SNR205.5+00.5': '/Users/jmcohen/Desktop/cur/P7rep/SNR205.5+00.5/1gevTo316gev/SNR205.5+00.5.dat'}

In [300]:
os.chdir('/Users/jmcohen/Desktop/cur/P7rep/3degAdd_fixedDiff')
{os.path.split(os.path.split(x)[0])[0]:os.path.join('/Users/jmcohen/Desktop/cur/P7rep/3degAdd_fixedDiff',x) for x in glob('SNR*/%s/SNR*.dat' %(energyDir)) if not 'moved' in x}

{'SNR015.4+00.1': '/Users/jmcohen/Desktop/cur/P7rep/3degAdd_fixedDiff/SNR015.4+00.1/1gevTo316gev/SNR015.4+00.1_bu.dat',
 'SNR027.8+00.6': '/Users/jmcohen/Desktop/cur/P7rep/3degAdd_fixedDiff/SNR027.8+00.6/1gevTo316gev/SNR027.8+00.6.dat',
 'SNR028.8+01.5': '/Users/jmcohen/Desktop/cur/P7rep/3degAdd_fixedDiff/SNR028.8+01.5/1gevTo316gev/SNR028.8+01.5.dat',
 'SNR034.7-00.4': '/Users/jmcohen/Desktop/cur/P7rep/3degAdd_fixedDiff/SNR034.7-00.4/1gevTo316gev/SNR034.7-00.4.dat',
 'SNR049.2-00.7': '/Users/jmcohen/Desktop/cur/P7rep/3degAdd_fixedDiff/SNR049.2-00.7/1gevTo316gev/SNR049.2-00.7.dat',
 'SNR073.9+00.9': '/Users/jmcohen/Desktop/cur/P7rep/3degAdd_fixedDiff/SNR073.9+00.9/1gevTo316gev/SNR073.9+00.9.dat',
 'SNR074.0-08.5': '/Users/jmcohen/Desktop/cur/P7rep/3degAdd_fixedDiff/SNR074.0-08.5/1gevTo316gev/SNR074.0-08.5.dat',
 'SNR089.0+04.7': '/Users/jmcohen/Desktop/cur/P7rep/3degAdd_fixedDiff/SNR089.0+04.7/1gevTo316gev/SNR089.0+04.7.dat',
 'SNR127.1+00.5': '/Users/jmcohen/Desktop/cur/P7rep/3degAdd_f

# Which of the 55 SNRs are in 3FGL/2FHL/elsewhere?

In [318]:
fgl3ES = newFglDF[newFglDF.Extended_Source_Name.notnull()]
fgl3ES.Extended_Source_Name #These are not all SNRs

Source_Name
3FGL J0059.0-7242e                SMC
3FGL J0526.6-6825e                LMC
3FGL J0540.3+2756e              S 147
3FGL J0617.2+2234e              IC443
3FGL J0822.6-4250e           Puppis A
3FGL J0833.1-4511e             Vela X
3FGL J0852.7-4631e            Vela Jr
3FGL J1303.0-6312e     HESS J1303-631
3FGL J1324.0-4330e         CenA Lobes
3FGL J1514.0-5915e          MSH 15-52
3FGL J1615.3-5146e     HESS J1614-518
3FGL J1616.2-5054e     HESS J1616-508
3FGL J1633.0-4746e     HESS J1632-478
3FGL J1713.5-3945e    RX J1713.7-3946
3FGL J1801.3-2326e                W28
3FGL J1805.6-2136e                W30
3FGL J1824.5-1351e     HESS J1825-137
3FGL J1836.5-0655e     HESS J1837-069
3FGL J1840.9-0532e     HESS J1841-055
3FGL J1855.9+0121e                W44
3FGL J1923.2+1408e               W51C
3FGL J2021.0+4031e        gamma Cygni
3FGL J2028.6+4110e      Cygnus Cocoon
3FGL J2045.2+5026e              HB 21
3FGL J2051.0+3040e        Cygnus Loop
Name: Extended_Source_Name, dtype: obj

### The new 2FHL ES are W41, kookaburra, HESS J1356, NGC xxxx, and RCW 86
### there was also the source near kookaburra (not associated with an SNR?) and Helene's SNR
### what about the other SNRs waiting to be published CTB 109, HESS J1731−347 (SNR G353.6-0.7), SN 1006 ( both young!), G326.31.8 (fabio's new composite source) , CTB 109, PKS 1209-51/52, CTB 37A
